# Reading Toshiba Data

In [ ]:
import os
import sys
import glob
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Local modules
import netCDF4

# iRadar
sys.path.append(os.path.expanduser('~/Developer/iradar/python'))
import iradar

# Get the First File

In [ ]:
files = glob.glob(os.path.expanduser('~/Downloads/toshiba/*.nc'))
filename = files[0]

azimuth_count = 360
range_count = 800

In [ ]:
print(filename)

# Read the File

In [ ]:
fid = netCDF4.Dataset(filename, 'r')

# Attributes, dimensions, and variables
atts = fid.ncattrs()
ds = [d for d in fid.dimensions]
vs = [v for v in fid.variables]
print(vs)

# Get the variables that I care about
lat = float(fid.variables['siteLat'][0])
lon = float(fid.variables['siteLon'][0])
start_time = time.gmtime(int(fid.variables['esStartTime'][0]))

scan_el = np.array(fid.variables['elevationAngle'][0], dtype=np.float)
ee = np.array(fid.variables['radialElev'])
aa = np.array(fid.variables['radialAzim'])
zz = fid.variables['Z']
print(zz)
print(zz['scale_factor'])

r0 = float(fid.variables['firstGateRange'][0])
dr = float(fid.variables['gateSize'][0])
ng = zz.shape[1]
rr = np.arange(r0, ng * dr, dr)

m = zz.mask
print(m)
z = np.array(zz)
v = np.zeros(z.shape)
z[m] = np.nan
v[m] = np.nan

if not zz.shape[0] == len(aa):
    print('Inconsistent azimuth count!  {} / {}'.format(z.shape[0], len(aa)))
    
fid.close()

# Construct a sweep dictionary

In [ ]:
# o = {
#         'longitude': longitude,
#         'latitude': latitude,
#         'time': sweep_time,
#         'ee': ee,
#         'aa': aa,
#         'rr': rr,
#         'z': zs,
#         'v': vs,
#         'm': ms
#     }

sweep = {
    'longitude': lon,
    'latitude': lat,
    'time': start_time,
    'scan_el': scan_el,
    'ee': ee,
    'aa': aa,
    'rr': rr,
    'z': z,
    'v': v,
    'm': m
}

print(sweep)

In [ ]:
fid = netCDF4.Dataset(filename, 'r')

# Attributes, dimensions, and variables
atts = fid.ncattrs()
ds = [d for d in fid.dimensions]
vs = [v for v in fid.variables]
print(vs)

# Get the variables that I care about
lat = float(fid.variables['siteLat'][0])
lon = float(fid.variables['siteLon'][0])
start_time = time.gmtime(int(fid.variables['esStartTime'][0]))

scan_el = np.array(fid.variables['elevationAngle'][0], dtype=np.float)
ee = np.array(fid.variables['radialElev'])
aa = np.array(fid.variables['radialAzim'])
zz = fid.variables['Z']

In [ ]:
zz